In [64]:
import json
import random
import os
from PIL import Image, ImageOps
from itertools import product
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

In [65]:
# symbol_images_dir = '../data/symbol_images/'
symbol_images_dir = './fewshot/'

def render_img(img_paths):
    # print(img_paths)
    images = [Image.open(symbol_images_dir + x.split('/')[0] + '.jpg') for x in img_paths]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('L', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]
    return new_im

In [66]:
dataset_ori = json.load(open('../data/fewshot_dataset.json'))

In [67]:
train_set = []
test_set = []

max_expr_len = 5
min_expr_len = 1
n_per_eval = 1
eval_splits = ['I', 'SS', 'LS', 'SL', 'LL']

for fs, data in dataset_ori.items():
	eval2data = {x: [] for x in eval_splits}
	for x in data['test']:
		eval2data[x['eval']].append(x)
	train_data_all = [x for x in data['train'] if len(x['expr']) >= min_expr_len and  len(x['expr']) <= max_expr_len]
	train_data = []
	while not train_data:
		train_data = random.sample(train_data_all, k=10)
		tmp = len([1 for x in train_data if len(x['expr']) == 3]) / len(train_data)
		if fs in 'abcd' and tmp > 0.3:
			train_data = []

	test_data = []
	tmp = random.sample([x for x in eval2data['SS'] if len(x['expr']) <= max_expr_len], n_per_eval)
	test_data.extend(tmp)
	tmp = random.sample([x for x in eval2data['SS'] if len(x['expr']) > max_expr_len and len(x['expr']) <= max_expr_len + 4], n_per_eval)
	test_data.extend(tmp)
	tmp = random.sample([x for x in eval2data['SL'] if len(x['expr']) <= max_expr_len + 2], n_per_eval)
	test_data.extend(tmp)
	tmp = random.sample([x for x in eval2data['SL'] if len(x['expr']) > max_expr_len + 2 and len(x['expr']) <= max_expr_len + 4], n_per_eval)
	test_data.extend(tmp)

	train_set.extend(train_data)
	test_set.extend(test_data)
	print(fs, len(train_data), len(test_data))

x 10 4
y 10 4
a 10 4
b 10 4
c 10 4
d 10 4


In [68]:
train_exprs = list(set([(x['expr'], x['res']) for x in train_set if len(x['expr']) <= 20]))
test_exprs = list(set([(x['expr'], x['res']) for x in test_set]))

In [69]:
dataset = {'train': [], 'test': []}
for split in ['train', 'test']:
    samples = train_set if split == 'train' else test_set
    for i, sample in enumerate(tqdm(samples)):
        res = sample['res']
        expr_img = render_img(sample['img_paths'])
        # print(sample['expr'], res)
        # display(expr_img)
        # input()
        file_name = f'{split}_{i:08d}.jpg'
        expr_img.save('../docs/fewshot/images/' + file_name)
        dataset[split].append(['fewshot/images/' + file_name, res])
json.dump(dataset, open('../docs/fewshot/data.json', 'w'))

100%|██████████| 24/24 [00:00<00:00, 2334.17it/s]


In [71]:
print([x[1] for x in dataset['test']])

[13, 3, 132, 136, 20, 17, 192, 756, 28, 32, 512, 144, 0, 55, 6, 3, 0, 29, 257, 140, 0, 40, 10, 2]
